# Questions ansered in this code are

## 1) Which are the movies with the third-lowest and third-highest budget?
## 2) What is the average number of words in movie titles between the year 2000-2005?
## 3) What is the most common Genre for Vin Diesel & Emma Watson movies?
## 4) Which are the movies with most and least earned revenue?
## 5) What is the average runtime of movies in the year 2006?

Importing the liberaries to work with

In [ ]:
import pandas as pd

Importing the dataset

In [ ]:
dataset = pd.read_csv('../input/tmdb-movies-dataset/tmdb_movies_data.csv')
dataset = pd.DataFrame(dataset)
print(dataset.info())

here you can see there are  many columns we will not get much information from, so a cleaning will be performed!!

In [ ]:
dataset.drop(["id", "homepage", "keywords", "overview", "production_companies", "tagline", "imdb_id", "popularity", "director", "vote_count", "vote_average", "budget_adj", "revenue_adj", "release_year"], axis = 1, inplace = True)
dataset.drop_duplicates(inplace=True)
dataset.drop(dataset[dataset['budget'] == 0].index, inplace = True) 
dataset.drop(dataset[dataset['revenue'] == 0].index, inplace = True) 
dataset.insert(0, 'New_ID', range(0, len(dataset)))
print(dataset.info())

Now the dataset is much clean and can be used properly to get oour ansers!

In [ ]:
print("Dates beofre processing them\n", dataset['release_date'].head(5))
dataset['release_date'] = pd.to_datetime(dataset['release_date'])
print("Dates after processing them\n", dataset['release_date'].head(5))

now it's time to solve our questions.
# Which are the movies with the third-lowest and third-highest budget?

In [ ]:
def sort(column, n_num, keep_type_max, keep_type_min, keep_all): ############# column = dataset['column'] ### n_num, keep_type_max, keep_type_min, keep_all = nLargest or nSmallest parameters
    dataset_sort = dataset.copy()
    if(keep_all == "no"): 
        dataset_sort = dataset_sort.drop_duplicates(subset = column, keep="first")
    maximum = dataset_sort.loc[dataset_sort[column].nlargest(n_num, keep = keep_type_max).idxmin()]
    id_max = dataset_sort.loc[lambda dataset_sort: dataset_sort[column] == maximum[column]]
    
    minimum = dataset_sort.loc[dataset_sort[column].nsmallest(n_num, keep = keep_type_min).idxmax()]
    id_min = dataset_sort.loc[lambda dataset_sort: dataset_sort[column] == minimum[column]]
    return id_max['original_title'], id_min['original_title']

maximum, minimum = sort('budget', 3, "all", "all", "no")
print(str("The third most budget movies in dataset are :- \n" + str(maximum) + "\n\nThe third least budget movies in dataset are :- \n" + str(minimum) + "\n\n############################################################\n"))

# What is the average number of words in movie titles between the year 2000-2005?

In [ ]:
def average(start, end, column_use, column_want, text): ############# steat, end = limits for average ### column_use, column_want = dataset['column'] ### text = the working variable for average is text or number
    identity_for_year = dataset.loc[lambda dataset: ((dataset[column_use] > start) & (dataset[column_use] < end))]
    sum = 0
    for temp in identity_for_year[column_want]:
        if (text == "text"):
            pieces = len(temp.split())
            sum = sum + pieces
        if (text == "number"):
            sum = sum + temp
    average_count = sum/len(identity_for_year)
    return average_count

count = average(pd.datetime(1999,12,31), pd.datetime(2007,1,1), 'release_date', 'original_title', "text")         
print(str("Average Word count in movie titles in 2000 - 2006 is approx :- \n" + str(round(count, 1)) + " Words\n\n############################################################\n"))

# What is the most common Genre for Vin Diesel & Emma Watson movies?

In [ ]:
def tit_for_tat(data1, column2, data1_column):
    dataset_temp = dataset.copy()
    dataset_temp[data1_column] = dataset_temp[data1_column].str.contains(data1)
    dataset_temp.drop(dataset_temp[dataset_temp[data1_column] == False].index, inplace = True)
    dataset_temp.drop(["New_ID", "budget", "revenue", "original_title", "runtime", "release_date"], axis = 1, inplace = True)
    dataset_temp.insert(0, 'New_ID', range(0, len(dataset_temp)))
    
    set_genre = set()
    for temp_genre in dataset_temp[column2].str.split('|').values:
        if isinstance(temp_genre, float): continue
        set_genre = set_genre.union(temp_genre)
        
    genre_count = dict()
    for s in set_genre: genre_count[s] = 0
    for temp_genre in dataset_temp[column2].str.split('|'):        
        if type(temp_genre) == float and pd.isnull(temp_genre): continue        
        for s in [s for s in temp_genre if s in set_genre]: 
            if pd.notnull(s): genre_count[s] += 1
    genre_occurences = []
    for k,v in genre_count.items():
        genre_occurences.append([k,v])
    genre_occurences.sort(key = lambda x:x[1], reverse = True)
    return genre_occurences, genre_count

count, count1 = tit_for_tat("Vin Diesel", 'genres', 'cast')
print(str("The most common genre of movies performed by Vin Diesel is :- \n" + str(count[:1]) + "\n\n"))
count, count1 = tit_for_tat("Emma Watson", 'genres', 'cast')
print(str("The most common genre of movies performed by Emma Watson is :- \n" + str(count[:1]) + "\n\n############################################################\n"))

# Which are the movies with most and least earned revenue?

In [ ]:
maximum, minimum = sort('revenue', 1, "all", "all", "yes") ######################################## Using the fumction "sort" from question 1
print(str("The most revenue movies in dataset is :- \n" + str(maximum) + "\n\nThe least revenue movies in dataset is :- \n" + str(minimum) + "\n\n############################################################\n"))

# What is the average runtime of movies in the year 2006?

In [ ]:
count = average(pd.datetime(2005, 12, 31), pd.datetime(2007, 1, 1), 'release_date', 'runtime', "number") ######################################## Using the fumction "average" from question 2
print(str("Average runtime of movies in 2006 is approx :- \n" + str(round(count, 2)) + " Minutes\n"))